# ConIndVar

- Concat Independent Variable
- 독립변수 합치기

- 대상 변수: 사용할 변수 목록.xlsx 참고


해야할 일

1. 결측값 대치
    1. 안 하기! XGBoost는 결측값에 강건하다
    2. sknn imputer 만들어서 쓰기 
    3. missingpy 라이브러리의 missforest 사용
    4. xgbimputer 라이브러리 쓰기!
        - 이 방법 쓰려면 좀 복잡할듯
    5. 원래 하던대로 하기(최빈값, 평균값 등등)

2. 이상치 제거...?
    1. 한다
    2. 안 한다

3. 데이터 가공하기. 고혈압 여부 등





In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc

# 기본 글꼴을 D2Coding ligature으로 변경
rc("font", family="D2Coding ligature")

In [123]:
print(plt.rcParams['font.family'])

['D2Coding ligature']


- 원본 데이터 불러오기

- GH_9622 데이터셋 기준으로 인덱스 조정해야함.

In [124]:
AS1_01_EXAMINEE = pd.read_csv("./Dataset/raw/AS1_01_EXAMINEE.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_02_GEN = pd.read_csv("./Dataset/raw/AS1_02_GEN.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_03_DRSM = pd.read_csv("./Dataset/raw/AS1_03_DRSM.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_04_ACTIVE = pd.read_csv("./Dataset/raw/AS1_04_ACTIVE.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_05_MEDIC = pd.read_csv("./Dataset/raw/AS1_05_MEDIC.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_06_DISEASE = pd.read_csv("./Dataset/raw/AS1_06_DISEASE.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_07_TREAT = pd.read_csv("./Dataset/raw/AS1_07_TREAT.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_08_DRUG = pd.read_csv("./Dataset/raw/AS1_08_DRUG.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_13_SLEEP = pd.read_csv("./Dataset/raw/AS1_13_SLEEP.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_16_DIET = pd.read_csv("./Dataset/raw/AS1_16_DIET.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_17_FFQWEIGHT = pd.read_csv("./Dataset/raw/AS1_17_FFQWEIGHT.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_18_BIOCHEM = pd.read_csv("./Dataset/raw/AS1_18_BIOCHEM.csv", index_col=0, encoding='utf-8', low_memory=False)
AS1_19_ANTHRO = pd.read_csv("./Dataset/raw/AS1_19_ANTHRO.csv", index_col=0, encoding='utf-8', low_memory=False)
GH = pd.read_csv("./Dataset/raw/GH_9622.csv", index_col=0, encoding='utf-8', low_memory=False) # 경훈이가 가공해준 식이패턴 데이터             


In [125]:
print(len(AS1_01_EXAMINEE), len(GH))

10030 9622


In [126]:
mask = AS1_01_EXAMINEE.index.isin(GH.index)

AS1_01_EXAMINEE = AS1_01_EXAMINEE[mask]
AS1_02_GEN = AS1_02_GEN[mask]
AS1_03_DRSM = AS1_03_DRSM[mask]
AS1_04_ACTIVE = AS1_04_ACTIVE[mask]
AS1_05_MEDIC = AS1_05_MEDIC[mask]
AS1_06_DISEASE = AS1_06_DISEASE[mask]
AS1_07_TREAT = AS1_07_TREAT[mask]
AS1_08_DRUG = AS1_08_DRUG[mask]
AS1_13_SLEEP = AS1_13_SLEEP[mask]
AS1_16_DIET = AS1_16_DIET[mask]
AS1_17_FFQWEIGHT = AS1_17_FFQWEIGHT[mask]
AS1_18_BIOCHEM = AS1_18_BIOCHEM[mask]
AS1_19_ANTHRO = AS1_19_ANTHRO[mask]

In [127]:
VAR_TO_USE = pd.read_csv("./Docs/var_to_use_2024-06-05-1330I.csv")
VAR_TO_USE

,binary,cnt,cath0,cath1
0,AS1_SEX,AS1_TOTALC,AS1_MARRYA,AS1_AGE
1,AS1_TIED,HOMA_IR,AS1_JOBB,AS1_EDUA
2,AS1_SLPAMSF,WAIST_AVG,AS1_DRINK,AS1_INCOME
3,HAS_HYPERTENSION,BMI,NaN,AS1_SMOKEA
4,HAS_DIABETES,KHEI90,NaN,AS1_HVSMAM
5,HAS_HYPERLIPIDEMI,AMED,NaN,AS1_PHYACTL
6,NaN,AMED_NONALC,NaN,AS1_PHYACTM
7,NaN,DASH,NaN,AS1_PHYACTH
8,NaN,PRAL,NaN,AS1_HEALTH
9,NaN,NEAP,NaN,AS1_RGMEALFQA


In [128]:
VAR_DF = pd.read_csv("./Docs/var_list.csv")
VAR_DF

,테이블명,변수명,변수명 변환,변수 유형,사용여부,가공여부
0,AS1_01_EXAMINEE,AS1_Sex,AS1_SEX,binary,NaN,NaN
1,AS1_01_EXAMINEE,AS1_Age,AS1_AGE,cnt,NaN,NaN
2,AS1_02_GEN,AS1_MarryA,AS1_MARRYA,cnt,NaN,NaN
3,AS1_02_GEN,AS1_JobB,AS1_JOBB,cath0,NaN,NaN
4,AS1_02_GEN,AS1_EduA,AS1_EDUA,cath1,NaN,NaN
5,AS1_02_GEN,AS1_Income,AS1_INCOME,cnt,NaN,NaN
6,AS1_03_DRSM,AS1_Drink,AS1_DRINK,cath0,NaN,NaN
7,AS1_03_DRSM,AS1_TotAlc,AS1_TOTALC,cnt,NaN,NaN
8,AS1_03_DRSM,AS1_SmokeA,AS1_SMOKEA,cath1,NaN,NaN
9,AS1_03_DRSM,AS1_HvSmAm,AS1_HVSMAM,cnt,NaN,NaN


## Data Engineering

- 결측값
- 새로운 변수 가공
- 스케일링?

In [129]:
result = pd.DataFrame(index=AS1_01_EXAMINEE.index)

### Imputation

해야할 일

- 77777 -> 0 또는 특정 값
- 99999 -> NaN

-----

NaN(결측값) 대치 방법 정리

- 기본적으로, 데이터 엔지니어링을 통해 도메인에 맞게 결측값 대치
    - Domain-based Imputation, Expert-driven Imputation

- 그 외에는 sequential K-nearest neighbor로 결측값 대치


In [130]:
AS1_01_EXAMINEE.replace(99999, np.nan, inplace=True)
AS1_02_GEN.replace(99999, np.nan, inplace=True)
AS1_03_DRSM.replace(99999, np.nan, inplace=True)
AS1_04_ACTIVE.replace(99999, np.nan, inplace=True)
AS1_05_MEDIC.replace(99999, np.nan, inplace=True)
AS1_06_DISEASE.replace(99999, np.nan, inplace=True)
AS1_07_TREAT.replace(99999, np.nan, inplace=True)
AS1_08_DRUG.replace(99999, np.nan, inplace=True)
AS1_13_SLEEP.replace(99999, np.nan, inplace=True)
AS1_16_DIET.replace(99999, np.nan, inplace=True)
AS1_17_FFQWEIGHT.replace(99999, np.nan, inplace=True)
AS1_18_BIOCHEM.replace(99999, np.nan, inplace=True)
AS1_19_ANTHRO.replace(99999, np.nan, inplace=True)

In [131]:
print(f"""
{AS1_01_EXAMINEE.isna().sum()}
{AS1_02_GEN.isna().sum()}
{AS1_03_DRSM.isna().sum()}
{AS1_04_ACTIVE.isna().sum()}
{AS1_05_MEDIC.isna().sum()}
{AS1_06_DISEASE.isna().sum()}
{AS1_07_TREAT.isna().sum()}
{AS1_08_DRUG.isna().sum()}
{AS1_13_SLEEP.isna().sum()}
{AS1_16_DIET.isna().sum()}
{AS1_17_FFQWEIGHT.isna().sum()}
{AS1_18_BIOCHEM.isna().sum()}
{AS1_19_ANTHRO.isna().sum()}
""")



AS1_AREA      0
AS1_EDATE1    0
AS1_SEX       0
AS1_AGE       0
dtype: int64
AS1_FAMNUM         9
AS1_HAND          14
AS1_SEAS          74
AS1_MARRYA        38
AS1_MARRYAETC      0
AS1_RELIG         23
AS1_RELIGETC       0
AS1_RELIGPRT      85
AS1_JOBB          40
AS1_EDUA          53
AS1_RSDYR         35
AS1_HOUSETY       13
AS1_HOUSEPS       48
AS1_INCOME       133
dtype: int64
AS1_DRINK          37
AS1_DRDUA         155
AS1_DRQMO         142
AS1_TAK           104
AS1_TAKFQA        122
AS1_TAKAMA        121
AS1_TAKGS         191
AS1_BEER          104
AS1_BEERFQA       129
AS1_BEERAMA       136
AS1_BEERGS        270
AS1_RICE          104
AS1_RICEFQA       105
AS1_RICEAMA       110
AS1_RICEGS        121
AS1_WINE          104
AS1_WINEFQA       114
AS1_WINEAMA       114
AS1_WINEGS        146
AS1_SOJU          104
AS1_SOJUFQA       155
AS1_SOJUAMA       176
AS1_SOJUGS        487
AS1_HLIQ          104
AS1_HLIQFQA       110
AS1_HLIQAMA       113
AS1_HLIQGS        129
AS1_TOTALC        223

In [132]:
# 딕셔너리 초기화
result_dict = {}

# 테이블명을 키로, 변수명 변환을 값으로 딕셔너리 생성
for i, row in VAR_DF.iterrows():
   table_name = row['테이블명']
   var_trans = row['변수명 변환']
   if table_name not in result_dict:
       result_dict[table_name] = [var_trans]
   else:
       result_dict[table_name].append(var_trans)

print(result_dict)

{'AS1_01_EXAMINEE': ['AS1_SEX', 'AS1_AGE'], 'AS1_02_GEN': ['AS1_MARRYA', 'AS1_JOBB', 'AS1_EDUA', 'AS1_INCOME'], 'AS1_03_DRSM': ['AS1_DRINK', 'AS1_TOTALC', 'AS1_SMOKEA', 'AS1_HVSMAM'], 'AS1_04_ACTIVE': ['AS1_PHYACTL', 'AS1_PHYACTM', 'AS1_PHYACTH'], 'AS1_05_MEDIC': ['AS1_HEALTH', 'AS1_TIED'], 'AS1_06_DISEASE': ['HAS_HYPERTENSION', 'HAS_DIABETES', 'HAS_HYPERLIPIDEMI'], 'AS1_13_SLEEP': ['AS1_SLPAMSF'], 'AS1_16_DIET': ['AS1_RGMEALFQA'], 'AS1_18_BIOCHEM': ['HOMA_IR'], 'AS1_19_ANTHRO': ['WAIST_AVG', 'BMI'], '경훈': ['KHEI90', 'AMED', 'AMED_NONALC', 'DASH', 'PRAL', 'NEAP', 'DII']}


In [133]:
# # result: 머신러닝 모델에 실제로 쓰일 독립변수만 담았다.
# 나중에 새로 result 만들어짐
result = pd.concat([AS1_01_EXAMINEE[["AS1_SEX", "AS1_AGE"]],
                    AS1_02_GEN[["AS1_MARRYA", "AS1_JOBB", "AS1_EDUA", "AS1_INCOME"]],
                    AS1_03_DRSM[['AS1_DRINK', 'AS1_SMOKEA', 'AS1_HVSMAM']], # 'AS1_TOTALC'를 GH에서 불러옴
                    AS1_04_ACTIVE[['AS1_PHYACTL', 'AS1_PHYACTM', 'AS1_PHYACTH']],
                    AS1_05_MEDIC[['AS1_HEALTH', 'AS1_TIED']],
                    AS1_13_SLEEP[['AS1_SLPAMSF']],
                    AS1_16_DIET[['AS1_RGMEALFQA']],
                    GH[['AS1_TOTALC', 'KHEI90', 'AMED', 'AMED_NONALC', 'DASH', 'PRAL', 'NEAP', 'DII']]],
                    axis=1)

#### 77777

In [134]:
count_77777 = (result == 77777).sum()

print(count_77777)

AS1_SEX             0
AS1_AGE             0
AS1_MARRYA          0
AS1_JOBB            0
AS1_EDUA            0
AS1_INCOME          0
AS1_DRINK           0
AS1_SMOKEA          0
AS1_HVSMAM       7096
AS1_PHYACTL         0
AS1_PHYACTM         0
AS1_PHYACTH         0
AS1_HEALTH          0
AS1_TIED            0
AS1_SLPAMSF         0
AS1_RGMEALFQA       0
AS1_TOTALC          0
KHEI90              0
AMED                0
AMED_NONALC         0
DASH                0
PRAL                0
NEAP                0
DII                 0
dtype: int64


In [135]:
AS1_03_DRSM['AS1_HVSMAM'].replace(77777, 0, inplace=True)

C:\Users\sh22h\AppData\Local\Temp\ipykernel_24464\650409227.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  AS1_03_DRSM['AS1_HVSMAM'].replace(77777, 0, inplace=True)


#### 99999 -> NaN


- AS1_02_GEN: AS1_MARRYA 결혼여부
    - 6: 기타
    - 6이거나 99999인 사람이 정상적인 결혼생활일까?
    - AS1_MARRYAETC을 보면 부인 가출, 행방불명, 동거 이렇게 되어있다.
    - 1(미혼) 또는 4(별거)로 대치
- AS1_02_GEN: AS1_JOBB 직업종류
    - 이건 어떻게 해야하나
- AS1_02_GEN: AS1_EDUA 교육수준
    - 이건 어떻게 해야하나
- AS1_02_GEN: AS1_INCOME 월평균수입
    - 이건 직업종류, 교육수준과 관련 있지 않을까
    - AS1_JOBB, AS1_EDUA 별로 평균으로 대치
- AS1_03_DRSM: AS1_DRINK 음주여부
    - ~~없는 거 보니 totalc / GH에서 걸러질 듯~~
    - ?
- AS1_03_DRSM: AS1_SMOKEA 흡연여부, AS1_HVSMAM 하루 흡연량
    - 이건 어떻게 해야하나
    - AS1_SMOKEA가 결측이면 AS1_HVSMAM도 결측
    - AS1_HVSMAM이 결측이면 AS1_SMOKEA가 2, 3 또는 99999
    - AS1_SMOKEA가 2의 평균, 3의 평균으로 대치
- AS1_04_ACTIVE: AS1_PHYACTL, AS1_PHYACTM, AS1_PHYACTH
    - 이건 어떻게 해야하나
- AS1_05_MEDIC: AS1_HEALTH 전반적인 건강상태
    - 이건 어떻게 해야하나
- AS1_05_MEDIC: AS1_TIED 최근 피곤한지
    - 이건 어떻게 해야하나
- AS1_13_SLEEP: AS1_SLPAMSF 수면시간 충분한지
    - 이건 어떻게 해야하나
- AS1_16_DIET: AS1_RGMEALFQA 하루 정규식사 횟수
    - 대충 3끼 먹는다고 가정..?
    - 3끼 다 챙겨먹는 사람이 결측값이 되려나

-----

- 결측값 있는 놈 없애볼까?
    - ~~GH에서 뺴면 얼마나 결측값이 사라지는지 관측해야할듯~~
        - 이미 GH로 뺸거임...
    - 결측값 있는 행 중에 심뇌혈관질환이 없다면?
        - 결측값 있는 행 1028
        - 심뇌혈관질환 있는 행 36
        - 전체 0    9726
        - 전체 1     301
-----

- 도메인 기반으로 값 일부 대치한 후
- SKNN 돌리기
    - AS1_02_GEN, AS1_03_DRSM, AS1_04_ACTIVE, AS1_05_MEDIC, AS1_13_SLEEP, AS1_16_DIET
    - 돌린 후 result 데이터프레임에 다시 집어넣기


In [136]:
missing_values_per_row = result.isna().sum(axis=1)[lambda x: x > 0]
missing_values_per_row

DIST_ID
NIH23B1110112     1
NIH23B1250466     1
NIH23B1533757     1
NIH23B1433593     1
NIH23B1997664     2
NIH23B1323049     1
NIH23B1940339     1
NIH23B1625798     1
NIH23B1631153     1
NIH23B1378671     1
NIH23B1663164     8
NIH23B1176860     5
NIH23B1357709     4
NIH23B1011782     1
NIH23B1045637     6
NIH23B1934734     3
NIH23B1743880     1
NIH23B1883484     3
NIH23B1849737     1
NIH23B1952341     1
NIH23B1906479     1
NIH23B1431547     1
NIH23B1115620     1
NIH23B1895817     1
NIH23B1829563     3
NIH23B1769778     1
NIH23B1723414     1
NIH23B1728406     1
NIH23B1687072     2
NIH23B1001715     2
NIH23B1063015     1
NIH23B1629908     1
NIH23B1545656     1
NIH23B1259011     1
NIH23B1417787     1
NIH23B1170241     3
NIH23B1746428     3
NIH23B1344568     1
NIH23B1797644     1
NIH23B1671180     1
NIH23B1422129     1
NIH23B1455172     1
NIH23B1891085     1
NIH23B1109003     1
NIH23B1940362     1
NIH23B1108795     1
NIH23B1964886     3
NIH23B1454697     1
NIH23B1477235     1
NIH23B140129

In [137]:
count_nul = result.isna().sum()[lambda x: x > 0]

In [138]:
count_nul

AS1_MARRYA        38
AS1_JOBB          40
AS1_EDUA          53
AS1_INCOME       133
AS1_DRINK         37
AS1_SMOKEA        84
AS1_HVSMAM       140
AS1_PHYACTL      130
AS1_PHYACTM      195
AS1_PHYACTH      156
AS1_HEALTH        28
AS1_TIED          99
AS1_SLPAMSF       50
AS1_RGMEALFQA     63
dtype: int64

```
AS1_MARRYA        38
AS1_JOBB          40
AS1_EDUA          53
AS1_INCOME       133
AS1_DRINK         37
AS1_SMOKEA        84
AS1_HVSMAM       140
AS1_PHYACTL      130
AS1_PHYACTM      195
AS1_PHYACTH      156
AS1_HEALTH        28
AS1_TIED          99
AS1_SLPAMSF       50
AS1_RGMEALFQA     63
```

In [139]:
len(result)

9622

#### Domain-based

In [140]:
# AS1_MARRYA
AS1_02_GEN['AS1_MARRYA'] = AS1_02_GEN['AS1_MARRYA'].replace(6, 1).fillna(1)

In [141]:
# 'AS1_HVSMAM' 결측값 대치를 위한 함수 정의
def fill_missing_hvsmam(row, mean_dict):
    if pd.isnull(row['AS1_HVSMAM']):
        if not pd.isnull(row['AS1_SMOKEA']):
            smoke_value = row['AS1_SMOKEA']
            return mean_dict.get(smoke_value, row['AS1_HVSMAM'])
    return row['AS1_HVSMAM']

# 'AS1_SMOKEA' 값에 따른 'AS1_HVSMAM' 평균 계산
mean_dict = AS1_03_DRSM.groupby('AS1_SMOKEA')['AS1_HVSMAM'].mean().to_dict()

# 결측값 대치
AS1_03_DRSM['AS1_HVSMAM'] = AS1_03_DRSM.apply(lambda row: fill_missing_hvsmam(row, mean_dict), axis=1)

#### Sequential KNNImputer

In [142]:
from sklearn.impute import KNNImputer

In [143]:
# # Sequential KNNImputer
# def sequential_knn_impute(df, n_neighbors=3):
#     imputer = KNNImputer(n_neighbors=n_neighbors)
#     df_imputed = df.copy()
#     # Iterate until all NaNs are filled
#     while df_imputed.isna().sum().sum() > 0:
#         df_imputed = pd.DataFrame(imputer.fit_transform(df_imputed), columns=df.columns)
#     return df_imputed

# Sequential KNNImputer with specific columns
# def sequential_knn_impute(df, columns, n_neighbors=3):
#     imputer = KNNImputer(n_neighbors=n_neighbors)
#     df_imputed = df.copy()
    
#     # Apply imputation only to specified columns
#     subset = df_imputed[columns]
    
#     # Iterate until all NaNs in the specified columns are filled
#     while subset.isna().sum().sum() > 0:
#         subset = pd.DataFrame(imputer.fit_transform(subset), columns=columns, index=df.index)
#         df_imputed[columns] = subset
    
#     return df_imputed

# Sequential KNNImputer with exception handling for non-numeric columns
def sequential_knn_impute(df, n_neighbors=3):
    imputer = KNNImputer(n_neighbors=n_neighbors)
    df_imputed = df.copy()
    
    # Iterate over each column
    for column in df.columns:
        if df[column].dtype in [np.float64, np.int64]:
            subset = df_imputed[[column]]
            # Iterate until all NaNs in the specified column are filled
            while subset.isna().sum().sum() > 0:
                try:
                    subset = pd.DataFrame(imputer.fit_transform(subset), columns=[column], index=df.index)
                    df_imputed[column] = subset
                except ValueError:
                    print(f"Skipping column {column} due to incompatible data type.")
                    break

    return df_imputed

In [144]:
# # KNNImputer를 사용하여 결측값 대치

AS1_02_GEN = sequential_knn_impute(AS1_02_GEN)
AS1_03_DRSM = sequential_knn_impute(AS1_03_DRSM)
AS1_04_ACTIVE = sequential_knn_impute(AS1_04_ACTIVE)
AS1_05_MEDIC = sequential_knn_impute(AS1_05_MEDIC)
AS1_13_SLEEP = sequential_knn_impute(AS1_13_SLEEP)
AS1_16_DIET = sequential_knn_impute(AS1_16_DIET)

#### Impitation 결과

In [145]:
# result: 위 과정을 거친 것들 다시 모아서 만들기
result = pd.concat([AS1_01_EXAMINEE[["AS1_SEX", "AS1_AGE"]],
                    AS1_02_GEN[["AS1_MARRYA", "AS1_JOBB", "AS1_EDUA", "AS1_INCOME"]],
                    AS1_03_DRSM[['AS1_DRINK', 'AS1_SMOKEA', 'AS1_HVSMAM']], # 'AS1_TOTALC'를 GH에서 불러옴
                    AS1_04_ACTIVE[['AS1_PHYACTL', 'AS1_PHYACTM', 'AS1_PHYACTH']],
                    AS1_05_MEDIC[['AS1_HEALTH', 'AS1_TIED']],
                    AS1_13_SLEEP[['AS1_SLPAMSF']],
                    AS1_16_DIET[['AS1_RGMEALFQA']],
                    GH[['AS1_TOTALC', 'KHEI90', 'AMED', 'AMED_NONALC', 'DASH', 'PRAL', 'NEAP', 'DII']]],
                    axis=1)

####  범주형 변수 값을 int로 바꾸기

In [146]:
VAR_TO_USE.T

,0,1,2,3,4,5,6,7,8,9,10,11,12
binary,AS1_SEX,AS1_TIED,AS1_SLPAMSF,HAS_HYPERTENSION,HAS_DIABETES,HAS_HYPERLIPIDEMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cnt,AS1_TOTALC,HOMA_IR,WAIST_AVG,BMI,KHEI90,AMED,AMED_NONALC,DASH,PRAL,NEAP,DII,RC,AI
cath0,AS1_MARRYA,AS1_JOBB,AS1_DRINK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cath1,AS1_AGE,AS1_EDUA,AS1_INCOME,AS1_SMOKEA,AS1_HVSMAM,AS1_PHYACTL,AS1_PHYACTM,AS1_PHYACTH,AS1_HEALTH,AS1_RGMEALFQA,NaN,NaN,NaN


In [147]:
# 변환할 칼럼 리스트
columns_to_convert = [
    'AS1_SEX', 'AS1_TIED', 'AS1_SLPAMSF', 'AS1_MARRYA', 'AS1_JOBB', 'AS1_DRINK',
    'AS1_AGE', 'AS1_EDUA', 'AS1_INCOME', 'AS1_SMOKEA', 'AS1_HVSMAM', 
    'AS1_PHYACTL', 'AS1_PHYACTM', 'AS1_PHYACTH', 'AS1_HEALTH', 'AS1_RGMEALFQA'
]

# 각 칼럼의 값을 int 자료형으로 변환
result[columns_to_convert] = result[columns_to_convert].astype(int)

In [148]:
result.head()

,AS1_SEX,AS1_AGE,AS1_MARRYA,AS1_JOBB,AS1_EDUA,AS1_INCOME,AS1_DRINK,AS1_SMOKEA,AS1_HVSMAM,AS1_PHYACTL,...,AS1_SLPAMSF,AS1_RGMEALFQA,AS1_TOTALC,KHEI90,AMED,AMED_NONALC,DASH,PRAL,NEAP,DII
DIST_ID,,,,,,,,,,,,,,,,,,,,,
NIH23B1298125,1,53,2.0,2,3,5,3,1,0.0,1,...,2,3.0,23.15,61,6,5,30,125.648516,25.068314,2.062933
NIH23B1159376,2,44,2.0,5,1,2,1,0,0.0,7,...,2,2.0,0.00,35,3,3,24,119.929227,20.677190,2.699810
NIH23B1785393,1,47,2.0,3,1,2,3,3,8.0,2,...,1,3.0,23.57,38,3,2,15,141.011119,42.987319,1.297841
NIH23B1463054,2,43,2.0,3,1,2,3,0,0.0,2,...,1,3.0,0.29,51,2,2,21,141.744284,43.921116,1.274731
NIH23B1751168,1,61,2.0,3,3,1,3,3,20.0,0,...,2,3.0,43.41,27,4,4,19,127.987263,28.310385,1.615857


### Data 가공하기

- 새로 만든 데이터
    - 고혈압여부: HAS_HYPERTENSION
    - 당뇨여부: HAS_DIABETES
    - 고지혈증: HAS_HYPERLIPIDEMI
    - LDL: LDL
    - H0MA_IR: H0MA_IR
    - 허리둘레: WAIST_AVG
    - BMI: BMI

- 데이터 만들 때 사용한 변수도 결측값을 모두 대치한 다음 사용하자
    - 결측값있는 가공데이터 목록
        - HOMA_IR      280
        - WAIST_AVG      8
        - BMI            4
        - RC             3
        - AI             2 
    - HOMA_IR
        - AS1_18_BIOCHEM['AS1_GLU0_TR']
        - AS1_18_BIOCHEM['AS1_INS0']
    - WAIST_AVG
        - AS1_19_ANTHRO[['AS1_WAIST1', 'AS1_WAIST2', 'AS1_WAIST3']].mean(axis=1)
    - BNI
        - AS1_19_ANTHRO['AS1_HEIGHT']
        - AS1_19_ANTHRO['AS1_WEIGHT']
    - RC, AI (AS1_18_BIOCHEM)
        - AS1_18_BIOCHEM['AS1_TCHL_TR']
        - AS1_18_BIOCHEM['AS1_HDL_TR']
        - AS1_18_BIOCHEM['AS1_LDL_TR']
            - AS1_18_BIOCHEM['AS1_TG_TR']

- AS1_18_BIOCHEM, AS1_19_ANTHRO 두 데이터프레임을 통째로 impute

In [149]:
print(AS1_18_BIOCHEM.isna().sum())

AS1_GLU0_ORI           280
AS1_GLU0_TR            280
AS1_GLU60_ORI          489
AS1_GLU60_TR           489
AS1_GLU120_ORI         485
AS1_GLU120_TR          485
AS1_ALBUMIN_ORI          3
AS1_ALBUMIN_TR           3
AS1_BUN_ORI              2
AS1_BUN_TR               2
AS1_CREATININE_ORI       2
AS1_CREATININE_TR1       2
AS1_AST_ORI              2
AS1_AST_TR               2
AS1_ALT_ORI              3
AS1_ALT_TR               3
AS1_R_GTP_ORI            2
AS1_R_GTP_TR             2
AS1_TCHL_ORI             2
AS1_TCHL_TR              2
AS1_HDL_ORI              2
AS1_HDL_TR               2
AS1_TG_ORI               3
AS1_TG_TR                3
AS1_TOTPRT               2
AS1_TOTBIL               2
AS1_CA                   2
AS1_NA                   2
AS1_K                    2
AS1_CL                   2
AS1_CRP                  3
AS1_WBC                  2
AS1_RBC                  2
AS1_HB                   2
AS1_HCT                  2
AS1_HBA1C                3
AS1_PLAT                 2
A

전체 데이터프레임을 대치하면 다른 데이터에 왜곡이 되려나?
아니다 이미 결측값이 없었기 떄문이 큰 영향은 못 줄것이다.

In [150]:
AS1_18_BIOCHEM = sequential_knn_impute(AS1_18_BIOCHEM)
AS1_19_ANTHRO = sequential_knn_impute(AS1_19_ANTHRO)

In [151]:
processed_df = pd.DataFrame(index=AS1_01_EXAMINEE.index)
processed_df.head()

""
DIST_ID
NIH23B1298125
NIH23B1159376
NIH23B1785393
NIH23B1463054
NIH23B1751168


#### 고혈압여부: HAS_HYPERTENSION
> pdht == 2 or drughtcu == 2 or 3회평균_sys >= 140 or 3회평균_dia >=90

In [152]:
pdht = AS1_06_DISEASE[["AS1_PDHT"]]
drughtcu = AS1_08_DRUG[['AS1_DRUGHTCU']]
sys = AS1_19_ANTHRO[['AS1_BPLIE1S', 'AS1_BPLIE2S', 'AS1_BPLIE3S']].mean(axis=1)
dia = AS1_19_ANTHRO[['AS1_BPLIE1D', 'AS1_BPLIE2D', 'AS1_BPLIE3D']].mean(axis=1)

df = pd.concat([pdht, drughtcu], axis=1)
df["AS1_AVG_SYS"] = sys
df["AS1_AVG_DIA"] = dia
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9622 entries, NIH23B1298125 to NIH23B1160138
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   AS1_PDHT      9621 non-null   float64
 1   AS1_DRUGHTCU  9115 non-null   float64
 2   AS1_AVG_SYS   9622 non-null   float64
 3   AS1_AVG_DIA   9622 non-null   float64
dtypes: float64(4)
memory usage: 375.9+ KB


In [153]:
df["AS1_DRUGHTCU"].value_counts()

AS1_DRUGHTCU
77777.0    8003
2.0        1043
1.0          69
Name: count, dtype: int64

In [154]:
conditions = (
    (df['AS1_PDHT'] == 2) | 
    (df['AS1_DRUGHTCU'] == 2) | 
    (df['AS1_AVG_SYS'] >= 140) | 
    (df['AS1_AVG_DIA'] >= 90)
)

# df['HAS_HYPERTENSION'] = np.where(conditions, 1, 0)
processed_df['HAS_HYPERTENSION'] = conditions.astype(int)

In [155]:
processed_df['HAS_HYPERTENSION'].value_counts()

HAS_HYPERTENSION
0    7192
1    2430
Name: count, dtype: int64

#### 당뇨여부

> pddm == 2 or druginscu == 2 or glu0 >= 126 or hba1c >= 6.5 or glu120 >= 200

In [156]:
pddm = AS1_06_DISEASE[["AS1_PDDM"]]
druginscu = AS1_08_DRUG[['AS1_DRUGINSCU']]
glu0 = AS1_18_BIOCHEM[['AS1_GLU0_TR']]
glu120 = AS1_18_BIOCHEM[['AS1_GLU120_TR']]
hba1c = AS1_18_BIOCHEM[['AS1_HBA1C']]

df = pd.concat([pddm, druginscu, glu0, glu120, hba1c], axis=1)

In [157]:
conditions = (
    (df['AS1_PDDM'] == 2) | 
    (df['AS1_DRUGINSCU'] == 2) | 
    (df['AS1_GLU0_TR'] >= 126) | 
    (df['AS1_GLU120_TR'] >= 200) |
    (df['AS1_HBA1C'] >= 6.5)
)

processed_df['HAS_DIABETES'] = conditions.astype(int)

In [158]:
processed_df['HAS_DIABETES'].value_counts()

HAS_DIABETES
0    8237
1    1385
Name: count, dtype: int64

#### 고지혈증
> tchl >= 240 or LDL >= 160 or TG >= 200 or HDL < 40

- 현대인은 모두 고지혈증인가? 어떻게 2가 없지

In [159]:
pdlp = AS1_06_DISEASE[["AS1_PDLP"]]
drugslcu = AS1_08_DRUG[['AS1_DRUGSLCU']]
tchl = AS1_18_BIOCHEM[['AS1_TCHL_TR']]
hdl = AS1_18_BIOCHEM[['AS1_HDL_TR']]
tg = AS1_18_BIOCHEM[['AS1_TG_TR']]

cholesterol_df = pd.concat([pdlp, drugslcu, tchl, hdl, tg], axis=1)
cholesterol_df['AS1_LDL_TR'] = cholesterol_df['AS1_TCHL_TR'] - cholesterol_df['AS1_HDL_TR'] - cholesterol_df['AS1_TG_TR'] / 5

In [160]:
cholesterol_df['AS1_TG_TR']/5

DIST_ID
NIH23B1298125     39.60000
NIH23B1159376     62.00000
NIH23B1785393     66.80000
NIH23B1463054     21.00000
NIH23B1751168     19.00000
NIH23B1703619     11.60000
NIH23B1699060     48.00000
NIH23B1971042     27.20000
NIH23B1338120     60.60000
NIH23B1475963     45.40000
NIH23B1816289     23.80000
NIH23B1498753     20.60000
NIH23B1054423     20.60000
NIH23B1831908     32.40000
NIH23B1743677     41.80000
NIH23B1961177     67.40000
NIH23B1909485     22.20000
NIH23B1758148     22.40000
NIH23B1111465     19.00000
NIH23B1620823     44.20000
NIH23B1110112     23.60000
NIH23B1250466     25.40000
NIH23B1309271     53.80000
NIH23B1479282     18.00000
NIH23B1525476     37.40000
NIH23B1533757     28.80000
NIH23B1433593     17.20000
NIH23B1726793     20.20000
NIH23B1879696    124.80000
NIH23B1997664     27.00000
NIH23B1160975     22.00000
NIH23B1226704     22.20000
NIH23B1815817     16.80000
NIH23B1526683     26.60000
NIH23B1217067     41.40000
NIH23B1714692     26.20000
NIH23B1169285     30

In [161]:
cholesterol_df.head()

,AS1_PDLP,AS1_DRUGSLCU,AS1_TCHL_TR,AS1_HDL_TR,AS1_TG_TR,AS1_LDL_TR
DIST_ID,,,,,,
NIH23B1298125,1.0,77777.0,159.0,36.0,198.0,83.4
NIH23B1159376,1.0,NaN,156.0,30.0,310.0,64.0
NIH23B1785393,1.0,77777.0,161.0,44.0,334.0,50.2
NIH23B1463054,1.0,77777.0,161.0,40.0,105.0,100.0
NIH23B1751168,1.0,NaN,151.0,44.0,95.0,88.0


In [162]:
conditions = (
    (cholesterol_df['AS1_PDLP'] == 2) | 
    (cholesterol_df['AS1_DRUGSLCU'] == 2) | 
    (cholesterol_df['AS1_TCHL_TR'] >= 240) | 
    (cholesterol_df['AS1_LDL_TR'] >= 160) |
    (cholesterol_df['AS1_TG_TR'] >= 200) |
    (cholesterol_df['AS1_HDL_TR']) < 40
)

processed_df['HAS_HYPERLIPIDEMI'] = conditions.astype(int)


In [163]:
processed_df['HAS_HYPERLIPIDEMI'].value_counts()

HAS_HYPERLIPIDEMI
1    9622
Name: count, dtype: int64

#### HOMA_IR, 허리둘레, BMI
> HOMA_IR=fasting glucose (mg/dL) X fasting insulin (mU/L) / 405

- AS1_Glu0_TR: mg/dL
- AS1_Ins0: µIU/㎖

- AS1_INS0 µIU/㎖ == uU/mL == mU/L
    - µ == u // 일부 이렇게 표시하는 사람 있음
    - IU == U // 같은 거임
    - 둘은 같다. 근거: https://www.quora.com/Any-one-can-help-how-I-can-convert-between-uU-ml-to-mcunit-ml-for-insulin-measuring-to-use-it-in-HOMA-IR-calculation

In [164]:
# HOMA-IR
processed_df['HOMA_IR'] = (AS1_18_BIOCHEM['AS1_GLU0_TR'] * AS1_18_BIOCHEM['AS1_INS0']) / 405

In [165]:
pd.options.display.max_rows = None
# print(len(df[AS1_18_BIOCHEM['AS1_GLU0_TR'] == 99999]))
# print(len(df[AS1_18_BIOCHEM['AS1_INS0'] == 99999]))
print(AS1_18_BIOCHEM['AS1_GLU0_TR'].isna().sum())
print(AS1_18_BIOCHEM['AS1_INS0'].isna().sum())

0
0


In [166]:
# 허리둘레

processed_df['WAIST_AVG'] = AS1_19_ANTHRO[['AS1_WAIST1', 'AS1_WAIST2', 'AS1_WAIST3']].mean(axis=1)

In [167]:
# BMI

processed_df['BMI'] = ((AS1_19_ANTHRO['AS1_HEIGHT'] / 100) ** 2) * AS1_19_ANTHRO['AS1_WEIGHT']

In [168]:
processed_df['HOMA_IR'].describe()

count    9622.000000
mean        1.667639
std         1.246296
min         0.016543
25%         1.059259
50%         1.478025
75%         2.019136
max        53.906667
Name: HOMA_IR, dtype: float64

#### remnant cholesterol
remnant cholesterol = total cholesterol – high-density lipoprotein (HDL) cholesterol – LDL-C

In [169]:
cholesterol_df.head()

,AS1_PDLP,AS1_DRUGSLCU,AS1_TCHL_TR,AS1_HDL_TR,AS1_TG_TR,AS1_LDL_TR
DIST_ID,,,,,,
NIH23B1298125,1.0,77777.0,159.0,36.0,198.0,83.4
NIH23B1159376,1.0,NaN,156.0,30.0,310.0,64.0
NIH23B1785393,1.0,77777.0,161.0,44.0,334.0,50.2
NIH23B1463054,1.0,77777.0,161.0,40.0,105.0,100.0
NIH23B1751168,1.0,NaN,151.0,44.0,95.0,88.0


In [170]:
processed_df['RC'] = cholesterol_df["AS1_TCHL_TR"] - cholesterol_df["AS1_HDL_TR"] - cholesterol_df["AS1_LDL_TR"]

In [171]:
processed_df['RC'].describe()

count    9622.000000
mean       32.428860
std        20.956331
min         6.400000
25%        20.000000
50%        27.000000
75%        38.200000
max       311.400000
Name: RC, dtype: float64

#### 동맥경화지수(Atherogenic Index, AI)

- Fiordaliso의 계산법(Shin et al., 2010)
- AI = ([총콜레스테롤]-[HDL콜레스테롤]/ [HDL콜레스테롤) 

In [172]:
processed_df['AI'] = (cholesterol_df["AS1_TCHL_TR"] - cholesterol_df["AS1_HDL_TR"]) / cholesterol_df["AS1_HDL_TR"]

In [173]:
processed_df['AI'].describe()

count    9622.000000
mean        3.456727
std         1.132253
min        -0.200000
25%         2.631579
50%         3.356349
75%         4.169326
max        10.875000
Name: AI, dtype: float64

#### 가공 결과

In [174]:
count_nul = processed_df.isna().sum()[lambda x: x > 0]
count_nul

Series([], dtype: int64)

In [175]:
missing_values_per_row = processed_df.isna().sum(axis=1)[lambda x: x > 1]
missing_values_per_row

Series([], dtype: int64)

## 결과

In [176]:
result_df = pd.concat([result, processed_df], axis=1)

In [177]:
result_df.head()

,AS1_SEX,AS1_AGE,AS1_MARRYA,AS1_JOBB,AS1_EDUA,AS1_INCOME,AS1_DRINK,AS1_SMOKEA,AS1_HVSMAM,AS1_PHYACTL,...,NEAP,DII,HAS_HYPERTENSION,HAS_DIABETES,HAS_HYPERLIPIDEMI,HOMA_IR,WAIST_AVG,BMI,RC,AI
DIST_ID,,,,,,,,,,,,,,,,,,,,,
NIH23B1298125,1,53,2.0,2,3,5,3,1,0.0,1,...,25.068314,2.062933,1,1,1,2.721235,91.933333,200.954880,39.6,3.416667
NIH23B1159376,2,44,2.0,5,1,2,1,0,0.0,7,...,20.677190,2.699810,0,0,1,0.306667,87.066667,135.594778,62.0,4.200000
NIH23B1785393,1,47,2.0,3,1,2,3,3,8.0,2,...,42.987319,1.297841,0,0,1,0.911111,80.333333,170.473543,66.8,2.659091
NIH23B1463054,2,43,2.0,3,1,2,3,0,0.0,2,...,43.921116,1.274731,0,0,1,1.274074,98.500000,170.236012,21.0,3.025000
NIH23B1751168,1,61,2.0,3,3,1,3,3,20.0,0,...,28.310385,1.615857,0,0,1,0.380247,80.666667,159.671232,19.0,2.431818


# 테스트

In [178]:
y_all = pd.read_csv("Dataset\y_all.csv", index_col=0)

In [179]:
y_all['CVD'].value_counts()

CVD
0    9726
1     301
Name: count, dtype: int64

In [180]:
y_all.head()

,AS1_PDMI,AS1_PDCH,AS1_PDCD,AS1_PDCV,AS1_TRTMI,AS1_TRTCH,AS1_TRTCD,AS1_TRTCV,AS1_DRUGSTKCU,CVD
DIST_ID,,,,,,,,,,
NIH23B1298125,1,1,1,1,1,1,1,1,77777,0
NIH23B1159376,1,1,1,1,99999,99999,99999,99999,99999,0
NIH23B1785393,1,1,1,1,99999,99999,99999,99999,77777,0
NIH23B1463054,1,1,1,1,1,1,1,1,77777,0
NIH23B1751168,1,1,1,1,99999,99999,99999,99999,99999,0


In [181]:
result_df.head()

,AS1_SEX,AS1_AGE,AS1_MARRYA,AS1_JOBB,AS1_EDUA,AS1_INCOME,AS1_DRINK,AS1_SMOKEA,AS1_HVSMAM,AS1_PHYACTL,...,NEAP,DII,HAS_HYPERTENSION,HAS_DIABETES,HAS_HYPERLIPIDEMI,HOMA_IR,WAIST_AVG,BMI,RC,AI
DIST_ID,,,,,,,,,,,,,,,,,,,,,
NIH23B1298125,1,53,2.0,2,3,5,3,1,0.0,1,...,25.068314,2.062933,1,1,1,2.721235,91.933333,200.954880,39.6,3.416667
NIH23B1159376,2,44,2.0,5,1,2,1,0,0.0,7,...,20.677190,2.699810,0,0,1,0.306667,87.066667,135.594778,62.0,4.200000
NIH23B1785393,1,47,2.0,3,1,2,3,3,8.0,2,...,42.987319,1.297841,0,0,1,0.911111,80.333333,170.473543,66.8,2.659091
NIH23B1463054,2,43,2.0,3,1,2,3,0,0.0,2,...,43.921116,1.274731,0,0,1,1.274074,98.500000,170.236012,21.0,3.025000
NIH23B1751168,1,61,2.0,3,3,1,3,3,20.0,0,...,28.310385,1.615857,0,0,1,0.380247,80.666667,159.671232,19.0,2.431818


In [182]:
print(len(result_df), len(y_all))

9622 10027


In [183]:
temp = pd.concat([result_df, y_all['CVD']], axis=1, join='inner')
temp.head()

,AS1_SEX,AS1_AGE,AS1_MARRYA,AS1_JOBB,AS1_EDUA,AS1_INCOME,AS1_DRINK,AS1_SMOKEA,AS1_HVSMAM,AS1_PHYACTL,...,DII,HAS_HYPERTENSION,HAS_DIABETES,HAS_HYPERLIPIDEMI,HOMA_IR,WAIST_AVG,BMI,RC,AI,CVD
DIST_ID,,,,,,,,,,,,,,,,,,,,,
NIH23B1298125,1,53,2.0,2,3,5,3,1,0.0,1,...,2.062933,1,1,1,2.721235,91.933333,200.954880,39.6,3.416667,0
NIH23B1159376,2,44,2.0,5,1,2,1,0,0.0,7,...,2.699810,0,0,1,0.306667,87.066667,135.594778,62.0,4.200000,0
NIH23B1785393,1,47,2.0,3,1,2,3,3,8.0,2,...,1.297841,0,0,1,0.911111,80.333333,170.473543,66.8,2.659091,0
NIH23B1463054,2,43,2.0,3,1,2,3,0,0.0,2,...,1.274731,0,0,1,1.274074,98.500000,170.236012,21.0,3.025000,0
NIH23B1751168,1,61,2.0,3,3,1,3,3,20.0,0,...,1.615857,0,0,1,0.380247,80.666667,159.671232,19.0,2.431818,0


In [184]:
# 결측값이 하나라도 있는 행 찾기
cvd_df = temp[temp.isna().any(axis=1)].copy()

# 각 행의 결측값의 수, 'CVD' 칼럼의 값을 데이터프레임으로 만들기
cvd_df['Missing_Values_Count'] = cvd_df.isna().sum(axis=1)
cvd_df = cvd_df[['Missing_Values_Count', 'CVD']]

In [185]:
len(cvd_df)

0

In [186]:
cvd_df.value_counts()

Series([], Name: count, dtype: int64)

In [187]:
mask_indices = cvd_df[cvd_df['CVD'] == 1].index
temp_filtered = temp.loc[mask_indices]

In [188]:
temp_filtered

,AS1_SEX,AS1_AGE,AS1_MARRYA,AS1_JOBB,AS1_EDUA,AS1_INCOME,AS1_DRINK,AS1_SMOKEA,AS1_HVSMAM,AS1_PHYACTL,...,DII,HAS_HYPERTENSION,HAS_DIABETES,HAS_HYPERLIPIDEMI,HOMA_IR,WAIST_AVG,BMI,RC,AI,CVD
DIST_ID,,,,,,,,,,,,,,,,,,,,,


In [189]:
temp.to_csv("./Dataset/result.csv")

In [190]:
temp['CVD'].value_counts()

CVD
0    9335
1     287
Name: count, dtype: int64

In [191]:
temp.describe()

,AS1_SEX,AS1_AGE,AS1_MARRYA,AS1_JOBB,AS1_EDUA,AS1_INCOME,AS1_DRINK,AS1_SMOKEA,AS1_HVSMAM,AS1_PHYACTL,...,DII,HAS_HYPERTENSION,HAS_DIABETES,HAS_HYPERLIPIDEMI,HOMA_IR,WAIST_AVG,BMI,RC,AI,CVD
count,9622.000000,9622.000000,9622.000000,9622.000000,9622.000000,9622.000000,9622.000000,9622.000000,9622.000000,9622.000000,...,9622.000000,9622.000000,9622.000000,9622.0,9622.000000,9622.000000,9622.000000,9622.000000,9622.000000,9622.000000
mean,1.523904,52.222927,2.101434,3.633028,2.349512,3.565163,2.012783,0.888485,4.466786,4.262523,...,2.135767,0.252546,0.143941,1.0,1.667639,82.641564,163.789382,32.428860,3.456727,0.029827
std,0.499454,8.909906,0.465078,2.422378,1.277502,1.920319,0.965221,1.227366,8.916095,2.481296,...,1.036805,0.434495,0.351048,0.0,1.246296,8.806334,39.862023,20.956331,1.132253,0.170120
min,1.000000,40.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,-13.160299,0.000000,0.000000,1.0,0.016543,55.666667,68.490842,6.400000,-0.200000,0.000000
25%,1.000000,44.000000,2.000000,1.000000,1.000000,2.000000,1.000000,0.000000,0.000000,2.000000,...,1.655135,0.000000,0.000000,1.0,1.059259,76.333333,133.997350,20.000000,2.631579,0.000000
50%,2.000000,50.000000,2.000000,3.000000,2.000000,3.000000,2.000000,0.000000,0.000000,4.000000,...,2.199585,0.000000,0.000000,1.0,1.478025,82.666667,158.795073,27.000000,3.356349,0.000000
75%,2.000000,60.000000,2.000000,5.000000,3.000000,5.000000,3.000000,2.000000,3.000000,6.000000,...,2.756392,1.000000,0.000000,1.0,2.019136,88.666667,189.763614,38.200000,4.169326,0.000000
max,2.000000,69.000000,5.000000,8.000000,6.000000,8.000000,3.000000,3.000000,80.000000,8.000000,...,5.611013,1.000000,1.000000,1.0,53.906667,122.666667,352.517500,311.400000,10.875000,1.000000


In [192]:
temp['AS1_INCOME'].value_counts()

AS1_INCOME
1    1824
5    1711
3    1603
2    1515
4    1316
6     937
7     528
8     188
Name: count, dtype: int64